In [6]:
import pandas as pd
import numpy as np
import urllib
import json                 
from pprint import pprint   
from urllib.request import Request, urlopen
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo 
import plotly
import seaborn as sns;
import category_encoders as ce  
import copy 
import matplotlib.pyplot



In [7]:
# Fetching data from API

req = Request('https://api.rootnet.in/covid19-in/stats/history', headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()
json_data = json.loads(webpage)
#print(json_data)
intermid1=json_data['data']
intermid2=[]
data=[]
print(len(intermid1))
for i in range(len(intermid1)):
   
    
    
    intermid2=intermid1[i]
    intermid2['summary']['TotalconfirmedCasesIndian'] = intermid2['summary'].pop('confirmedCasesIndian')
    intermid2['summary']['TotalconfirmedCasesForeign'] = intermid2['summary'].pop('confirmedCasesForeign')
    intermid2['summary']['Totaldischarged'] = intermid2['summary'].pop('discharged')
    intermid2['summary']['Totaldeaths'] = intermid2['summary'].pop('deaths')
    for j in range(len(intermid2['regional'])):
        intermid3 = intermid2['regional']
        summary = intermid2['summary']
        date = intermid2['day']
        for k in range(len(intermid3[j])):
            intermid4=intermid3[j]
            for l in summary:
                intermid4[l]=summary.get(l)
                intermid4['date']=date
        data.append(intermid4)
        
df = pd.DataFrame(data)
df.rename(columns={'loc':'STUT'}, inplace=True)






URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
json_data

In [ ]:
df['STUT'].head(24)

In [ ]:
# Encoding the states and union territories


ce_ordinal = ce.OrdinalEncoder(cols=['STUT'])
df=ce_ordinal.fit_transform(df)
df.head()

In [ ]:
# checking null values

df.isna().sum()

In [ ]:
# Fetching testing data from API

req2 = Request('https://api.rootnet.in/covid19-in/stats/testing/history', headers={'User-Agent': 'Mozilla/5.0'})

webpage2 = urlopen(req2).read()
json_data2 = json.loads(webpage2)['data']

df2 = pd.DataFrame(json_data2)
df2.rename(columns={'day':'date'}, inplace=True)




In [ ]:
df2.head()

In [ ]:
#display null values

df.isna().sum()

In [ ]:
# merging case and testing data based on date

df3 =  pd.merge(df, df2, on='date')
df3.head()


In [ ]:
df3.isna().sum()

In [ ]:
#Dropping tables with too many null values
df3 = df3.drop(['totalIndividualsTested','totalPositiveCases','source'],axis=1)


In [ ]:
df3.isna().sum()

In [ ]:
df4 = df3.copy()
df3['totalSamplesTested'][1]

In [ ]:
# replacing total samples tested with previous value wherever null
count=0
for i in range(df3['totalSamplesTested'].count()):
    if(np.isnan(df3['totalSamplesTested'][i])):
       df4['totalSamplesTested'][i]= df4['totalSamplesTested'][i-1]

In [ ]:
df4.isna().sum()


In [ ]:
# plotting heatmap of correlation matrix to visualize the dependency of one attribute over another

heatmap = df3
heatmap = heatmap.drop(columns="date")
heatmap.head()
sns.heatmap(heatmap.corr(), annot = True)

In [ ]:
# showing latest details

recent = intermid1[len(intermid1)-1]['regional']
recent  = pd.DataFrame(recent)

recent.rename(columns={'loc':'STUT'}, inplace=True)
recent.rename(columns={'totalConfirmed':'confirmed'}, inplace=True)

recent_x = recent.STUT



In [ ]:
recent.style.background_gradient(cmap='plasma_r')

### trace1 = {
  'x': recent_x,
  'y': recent.confirmed,
  'name': 'Confirmed',
  'type': 'bar'
};
trace2 = {
  'x': recent_x,
  'y': recent.discharged,
  'name': 'Discharged',
  'type': 'bar'
};

trace3 = {
  'x': recent_x,
  'y': recent.deaths,
  'name': 'Deaths',
  'type': 'bar'
};
data = [trace1, trace2,trace3];
layout = {
  'xaxis': {'title': ' State-Data '},
  'barmode': 'relative',
  'title': 'Case Wise Disturbution'
};

fig = go.Figure(data = data, layout = layout)
pyo.iplot(fig)





In [ ]:
df4['date'] = list(map(lambda x: pd.to_datetime(x), df4['date']))


In [ ]:
df4['date'][5000]

In [ ]:
subdf = df4[df4['STUT']==1].copy()
subdf.reset_index(inplace = True, drop = True) 
subdf

In [ ]:
subdf['newCases'] = subdf['confirmedCasesIndian']
subdf['newCases'][1]
subdf['newDeaths'] = subdf['Totaldeaths']

In [ ]:

subwithnew = subdf.copy()

for i in range(1,len(subdf)):
    #print(subwithnew['newCases'][0])
    subwithnew['newCases'][i] = subdf['newCases'][i]- subdf['newCases'][i-1]
    subwithnew['newDeaths'][i] = subdf['newDeaths'][i]- subdf['newDeaths'][i-1]


In [ ]:
subwithnew['newDeaths']

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(subwithnew['date'].values,period=10,  
                            model ='multiplicative') 
  
# ETS plot  
result.plot()

In [ ]:
subwithnew.plot('date','newDeaths')


In [ ]:
subwithnew=subwithnew.loc[:,['date','newDeaths']]
subwithnew=subwithnew[subwithnew['date']<'2020-10-20']
# index_no = subwithnew[(subwithnew['newDeaths']==0) & (subwithnew['date']>'2020-09-01')].index
# print(index_no)
# subwithnew.drop(index_no, inplace = True)
# subwithnew
subwithnew.plot('date','newDeaths')


In [ ]:
from pmdarima import auto_arima 
  
# Ignore harmless warnings 
import warnings 
warnings.filterwarnings("ignore") 
  
    
# Fit auto_arima function to AirPassengers dataset 
stepwise_fit = auto_arima(subwithnew['newDeaths'].astype(float), start_p = 1, start_q = 1, 
                          max_p = 3, max_q = 3, m = 12, 
                          start_P = 0, seasonal = True, 
                          d = None, D = 1, trace = True, 
                          error_action ='ignore',   # we don't want to know if an order does not work 
                          suppress_warnings = True,  # we don't want convergence warnings 
                          stepwise = True)           # set to stepwise 
  
# To print the summary 
stepwise_fit.summary() 

In [ ]:
subwithnew.count()

In [ ]:
# Split data into train / test sets 
train = subwithnew.iloc[:len(subwithnew)-30] 
test = subwithnew.iloc[len(subwithnew)-30:] # set one year(12 months) for testing 
#Fit a SARIMAX(0, 1, 1)x(2, 1, 1, 12) on the training set 
from statsmodels.tsa.statespace.sarimax import SARIMAX 
  
model = SARIMAX(train['newDeaths'].astype(float),  
                order = (0, 0, 0),  
                seasonal_order =(0, 1, 1, 12)) 
  
result = model.fit() 
result.summary()

In [ ]:
start = len(train) 
end = len(train) + len(test) - 1
  
# Predictions for one-year against the test set 
predictions = result.predict(start, end, 
                             typ = 'levels').rename("Predictions") 
  
# plot predictions and actual values 
predictions.plot(legend = True) 
test['newDeaths'].astype(float).plot(legend = True)